# Testing MQTT with the new version of json message

A new forma of array of data was necesary for canomed. Until some days the format used was exactly that one of the S10. However Canomed should not replicate that format and the need of a new own format was needed. Care should be taken to the new array of data, and the new position inside the array of the variable "sessions" that cont the number of sessions inside the register time.

In [1]:
import pandas as pd
import datetime
import json
from random import randrange

In [2]:
# READING OLD FORMAT CANNOMED STR: 10 sections position 4, multfactor=2
n_sec = 10
numb_sec_pos = 4
mask_on_pos = 2
n_factor = 1

Form = 'OLD'
donnees_root = 'Canomed_OldFormat'
donnees = pd.read_csv(f'{donnees_root}/STR_data.txt').reset_index()
donnees.drop(columns='Time', inplace= True)

In [3]:
# READING NEW FORMAT CANNOMED STR: 20 sections position 34, multfactor=1
n_sec = 20
numb_sec_pos = 35
mask_on_pos = 33
n_factor = 0

Form = 'NEW'
donnees_root = 'Canomed_NewFormat_20sections/STR4/'
donnees = pd.read_csv(f'{donnees_root}/STR_data.txt').reset_index()
donnees.drop(columns='Time', inplace= True)

## Creation  of the json with the data and sections

In [4]:
for i in range(0,10): #range(int(donnees.shape[0]/10)):    # loop in each daily section   
    xs = list(donnees.iloc[i*n_sec][1:])
    
    message = dict(data = xs)
    sessions = donnees.iloc[i*n_sec][numb_sec_pos]
    
    #print(sessions)
    if sessions >= 2: 
        if Form == 'OLD':
            j = (sessions-2)/2
        else:
            j = sessions
        for k in range(1,int(j)+n_factor):         
            S = f'SEC{k}'
            message[S]=[donnees.iloc[i*n_sec+k][mask_on_pos], donnees.iloc[i*n_sec+k][mask_on_pos+1]]
            
    print(json.dumps(message))  

{"data": [19887.0, 9.0, 1.0, 1.0, 10.0, 12.0, 5.0, 0.0, 1.0, 4.0, 2.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 16.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 593.0, 640.0, 2.0, 392.0, 0.0, 0.0, 0.0, 8.0, 8.0, 8.0, 5.84, 5.8, 5.74, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.22, 0.0, 9.875, 7.5, 6.75, 24.4, 18.4, 17.2, 0.62, 0.44, 0.38, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 71.0, 0.0, 0.0, 74.0, 0.0, 0.0, 0.0, 0.0, 79.0], "SEC1": [671.0, 1016.0]}
{"data": [19888.0, 9.0, 1.0, 1.0, 10.0, 12.0, 5.0, 0.0, 1.0, 4.0, 2.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 16.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 655.0, 982.0, 1.0, 327.0, 0.0, 0.0, 0.0, 8.0, 8.0, 8.0, 5.86, 5.8, 5.72, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24, 0.06, 0.0, 8.75, 6.875, 6.125, 19.4, 18.2, 16.2, 0.52, 0.42, 0.36, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 71.0, 0.0, 0.0, 74.0, 0.0, 0.0, 0.0, 0.0, 79.0]}
{"data": [19889.0, 9.0, 1.0, 1.0, 10.0, 12.0, 5.0, 0.0, 1.0, 4.0, 2.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.

C:\Users\piafo\AppData\Local\Temp\ipykernel_11124\936901242.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sessions = donnees.iloc[i*n_sec][numb_sec_pos]
C:\Users\piafo\AppData\Local\Temp\ipykernel_11124\936901242.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  message[S]=[donnees.iloc[i*n_sec+k][mask_on_pos], donnees.iloc[i*n_sec+k][mask_on_pos+1]]


## Creation of the formal Json MQTT message

In [ ]:
num_ser = ['CAN5501771', 'CAN5501772', 'CAN5501671', 'CAN5507895', 'CAN5507896', 
           'CAN5507897', 'CAN5507898', 'CAN5507899', 'CAN5507900', 'CAN5507901', 'CAN55079023']

for i in range(0,10): #range(int(donnees.shape[0]/10)):    # loop in each daily section
    
    # SERIAL NUMBER
    message = dict(SN = num_ser[randrange(10)])

    # DATE
    message['TM'] = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S')
    
    # STR DATA 
    xs = list(donnees.iloc[i*n_sec][1:])    
    message['DATA'] = xs
    
    # STR MASK SESSIONS
    sessions = donnees.iloc[i*n_sec][numb_sec_pos]   
    if sessions >=2:                         
        if Form == 'OLD':
            j = (sessions-2)/2
        else:
            j = sessions
        D = []
        for k in range(1,int(j)+n_factor):         
            D.append([donnees.iloc[i*n_sec+k][mask_on_pos], donnees.iloc[i*n_sec+k][mask_on_pos+1]])
        if D:
            message["MASK"] = D
    
    print(json.dumps(message))

## BIBLIOGRAPHY

In [ ]:
Data = [ "Date", "VersionSoft", "ModeTraitement", "ReactiviteTraitement", "ConsPressFixe", "ConsPressMax", "ConsPressMin", "ModeRampe", "ConsTempsRampe", "ConsPressDemarrage", "ModeSoftUse", "ConsSoftUse", "DemarrageAuto", "ArretAuto", "PresenceFiltre", "TypeMasque", "TypeTuyau", "PresenceHumidificateur", "HumidAuto", "ConsHumid", "PresenceCircuitChauffant", "ConsCircuitChauffant", "AccesPatientGeneral", "AccesPatientRampe", "AccesPatientSoftUse", "AccesPatientHumidificateur", "AccesPatientCircuitChauffant", "MoyHumidAmbiante", "MoyTempHumid", "MoyPuissHumid", "MoyTempCircuitChauffant", "MoyPuissCircuitChauffant", "HeureDebutSessionX", "HeureFinSessionX", "NbSession", "TempsUtilisation", "TempsUtilisationEff", "TempsUtilisationTotal", "TempsTurbine", "Press95", "Press50", "Press5", "PressMasqueMax", "PressMasque95", "PressMasque50", "PressNoFuites95", "PressNoFuites50", "Debit95", "Debit50", "Debit5", "FuitesMax", "Fuites95perc", "Fuites50perc", "VentMinuteMax", "VentMinute95", "VentMinute50", "FreqRespMax", "FreqResp95", "FreqResp50", "VolumeCourantMax", "VolumeCourant95", "VolumeCourant50", "IAH", "IA", "IH", "IAO", "IAC", "INoQua", "IHO", "IHC", "TempsRespCS", "TempsMoyCycleRCS", "NbRespMoyCycleRCS", "TempsRonflement", "PanneAppareil1", "PanneAppareil2", "PanneHumid", "PanneCircuitChauffant", "PanneEnregistrement"]

In [ ]:
Data.index('NbSession')

this is suposed to be the arrangement of data in the new format proposed y Gaetan. To notice that "sessions" now named "NbSession" is now in position 34 

Data = [    "Date",                     # Utiliser comme tag dans InfluxDB    "VersionSoft",    "ModeTraitement",    "ReactiviteTraitement",    "ConsPressFixe",    "ConsPressMax",    "ConsPressMin",    "ModeRampe",    "ConsTempsRampe",    "ConsPressDemarrage",    "ModeSoftUse",    "ConsSoftUse",    "DemarrageAuto",    "ArretAuto",    "PresenceFiltre",    "TypeMasque",    "TypeTuyau",    "PresenceHumidificateur",    "HumidAuto",    "ConsHumid",    "PresenceCircuitChauffant",    "ConsCircuitChauffant",    "AccesPatientGeneral",    "AccesPatientRampe",    "AccesPatientSoftUse",    "AccesPatientHumidificateur",    "AccesPatientCircuitChauffant",    "MoyHumidAmbiante",    "MoyTempHumid",    "MoyPuissHumid",    "MoyTempCircuitChauffant",    "MoyPuissCircuitChauffant",    "HeureDebutSessionX",    "HeureFinSessionX",    "NbSession",    "TempsUtilisation",    "TempsUtilisationEff",    "TempsUtilisationTotal",    "TempsTurbine",    "Press95",    "Press50",    "Press5",    "PressMasqueMax",    "PressMasque95",    "PressMasque50",    "PressNoFuites95",    "PressNoFuites50",    "Debit95",    "Debit50",    "Debit5",    "FuitesMax",    "Fuites95perc",    "Fuites50perc",    "VentMinuteMax",    "VentMinute95",    "VentMinute50",    "FreqRespMax",    "FreqResp95",    "FreqResp50",    "VolumeCourantMax",    "VolumeCourant95",    "VolumeCourant50",    "IAH",    "IA",    "IH",    "IAO",    "IAC",    "INoQua",    "IHO",    "IHC",    "TempsRespCS",    "TempsMoyCycleRCS",    "NbRespMoyCycleRCS",    "TempsRonflement",    "PanneAppareil1",    "PanneAppareil2",    "PanneHumid",    "PanneCircuitChauffant",    "PanneEnregistrement",]